In [2]:
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import glob
import pyLDAvis
import pyLDAvis.gensim_models
import itertools
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords



In [3]:
df = pd.read_csv('ibis_budget_review_sentiment.csv')
df

,Hotel_Name,Date,Title,Review,Rating,p_rev,polarity,Rating_adj,Pol_adj
0,ibis budget Singapore Pearl,April 2022,When they say budget they mean budget,When they say budget they really mean budget. ...,2,say budget really mean budget stay double room...,0.023264,Negative,Neutral
1,ibis budget Singapore Pearl,January 2021,Poor customer service,This hotel does not live up to expectation of ...,3,hotel not live expectation service standard ac...,-0.069697,Neutral,Negative
2,ibis budget Singapore Pearl,February 2020,It's okay,We stayed 4 nights. We got free room promotion...,3,stayed night get free room promotion double ro...,0.131111,Neutral,Neutral
3,ibis budget Singapore Pearl,February 2020,Average stay,Located in an area that looks a little more ru...,3,locate area look little run rest singapore fel...,0.221528,Neutral,Positive
4,ibis budget Singapore Pearl,January 2020,Good budget Hotel,Ibis pearl is good budget hotel in geylang are...,3,ibis pearl good budget hotel geylang area room...,0.400000,Neutral,Positive
...,...,...,...,...,...,...,...,...,...
5324,ibis budget Singapore Selegie,December 2016,"Accesible, but tiny rooms😓",This hotel was located near Orchard road and m...,3,hotel locate near orchard road midway little i...,-0.047917,Neutral,Neutral
5325,ibis budget Singapore Selegie,September 2017,F1 weekend,Hotel is close to really good restaurants and ...,4,hotel close really good restaurant local shop ...,0.336111,Positive,Positive
5326,ibis budget Singapore Selegie,November 2017,Amazing hotel in the heart of Singapore,"Beside the supper comfort, facilities and acco...",4,beside supper comfort facility accomodations h...,0.409722,Positive,Positive
5327,ibis budget Singapore Selegie,October 2017,Worth it <3,Great place to stay! ^_^ Walking distance to m...,5,great place stay walk distance mustafa one rid...,0.277083,Positive,Positive


- filtering out the processed reviews for the LDA model

In [4]:
df_rev = df[['p_rev']]
df_rev

,p_rev
0,say budget really mean budget stay double room...
1,hotel not live expectation service standard ac...
2,stayed night get free room promotion double ro...
3,locate area look little run rest singapore fel...
4,ibis pearl good budget hotel geylang area room...
...,...
5324,hotel locate near orchard road midway little i...
5325,hotel close really good restaurant local shop ...
5326,beside supper comfort facility accomodations h...
5327,great place stay walk distance mustafa one rid...


- split into individual words

In [5]:
split = []
for i in df_rev['p_rev']:
    s_text = i.split()
    split.append(s_text)

- initialise Dictionary method with list of split words

In [5]:
id2word = corpora.Dictionary(split)
id2word

- update corpus list where an id is attached to the word

In [6]:
corpus = []
for text in split:
    new=id2word.doc2bow(text)
    corpus.append(new)

- create the LDA model

In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, 
                                            id2word=id2word, 
                                            num_topics= 5, 
                                            random_state=123, 
                                            update_every=1,
                                            chunksize = 1000,
                                            passes = 20,
                                            alpha='auto')

- Vis the results

In [8]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds = 'mmds', R=30)
vis

c:\Users\edowi\anaconda3\envs\mages\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.174480 -0.146405       1        1  41.773174
2     -0.028620 -0.029156       2        1  29.473878
1     -0.148106 -0.255142       3        1  17.032497
0     -0.234053  0.200548       4        1   5.883038
3      0.236299  0.230155       5        1   5.837414, topic_info=          Term         Freq        Total Category  logprob  loglift
18       hotel  8805.000000  8805.000000  Default  30.0000  30.0000
29        room  8622.000000  8622.000000  Default  29.0000  29.0000
67        good  3140.000000  3140.000000  Default  28.0000  28.0000
34        stay  3643.000000  3643.000000  Default  27.0000  27.0000
196     shower  1266.000000  1266.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
310       best    94.377969   324.906422   Topic5  -5.0358   1.6047
85   singapore   145.464232  2312.148113   Topic5  -4.6032   0.0749
7        clean   138.452542  2964.769812   Topic5  -4.6526  -0.2231
157     around   108.373557  1274.565940   Topic5  -4.8976   0.3761
196     shower    83.373097  1266.628284   Topic5  -5.1598   0.1201

[380 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
429       1  0.993229  accessible
811       5  0.978171   advantage
1440      4  0.989451      afraid
2088      5  0.982668       agoda
521       2  0.994646       alone
...     ...       ...         ...
118       3  0.003538        work
118       5  0.120309        work
308       1  0.562315       would
308       2  0.385561       would
308       3  0.052202       would

[613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])